# Setup
Utilizaremos a biblioteca `boto3` para nos comunicarmos com a AWS.

Utilizaremos as bibliotecas `pandas` e `pyspark` para lidarmos com nossos dados.

Utilizaremos as bibliotecas `matplotlib` e `seaborn` para a análise final.

In [1]:
import pandas as pd
import boto3
import os

import matplotlib.pyplot as plt
import seaborn as sns

import findspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark import SparkContext

from sqlalchemy import create_engine, text
import pyarrow.parquet

In [2]:
BUCKET = '896309849144'
LAYERS_PREFIX = f'layers'# f's3://{BUCKET}/layers'
GOLD = f'{LAYERS_PREFIX}/gold'
SILVER = f'{LAYERS_PREFIX}/silver'
BRONZE = f'{LAYERS_PREFIX}/bronze'

LOCAL_BRONZE = './local/bronze'
LOCAL_SILVER = './local/silver'
LOCAL_GOLD = './local/gold'

files = [
    f'{LOCAL_BRONZE}/PNAD_COVID_092020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_102020.csv',
    f'{LOCAL_BRONZE}/PNAD_COVID_112020.csv']


In [3]:
s3_client = boto3.client('s3')

def upload_to_s3(
        file_path : str,
        bucket : str,
        bucket_path : str):
    '''
    Uploads file_path to the given bucket, as bucket_path (include the name of the file in bucket_path)
    
    Parameters
    -
    file_path : str
    bucket : str
    bucket_path : str
    '''
    s3_client.upload_file(file_path, bucket, bucket_path)

# 0. Ingestion
Faremos o upload dos dados brutos extraídos manualmente do IBGE, a um bucket AWS S3 via a biblioteca `boto3`.

Conforme o pedido, utilizaremos os últimos 3 meses disponíveis.

In [26]:
for file in files:
    print(file.split('/')[-1])

PNAD_COVID_092020.csv
PNAD_COVID_102020.csv
PNAD_COVID_112020.csv


# 1. Bronze
Subindo nossos dados para a AWS

In [ ]:
for file in files:
    upload_to_s3(
        file,
        BUCKET,
        f'{BRONZE}/{file.split("/")[-1]}')

O processo de re-estruturamento dos dados é feito na AWS via `AWS Glue`.

# 2. Silver
Pegaremos esses dados re-estruturados *silver* e faremos nossas queries via `PySpark`.

## Puxando nossos dados do S3

In [4]:
to_download = s3_client.list_objects_v2(
    Bucket = BUCKET,
    Prefix = SILVER
)['Contents'][1:]

count = 1

for file in to_download:
    s3_client.download_file(BUCKET, file['Key'], f'{LOCAL_SILVER}/silver_{count}.parquet')
    count+=1

## Checando integridade dos dados

Comparemos o tamanho da camade bronze original com a camada silver atual para nos certificarmos que o processo de merge ocorreu corretamente

In [5]:
df_silver = pd.read_parquet(f'{LOCAL_SILVER}')
df_silver.shape[0]

1149197

In [6]:
row_count = 0
for f in files[:]:
    _ = pd.read_csv(f)
    row_count += _.shape[0]

    print(_.shape[0])
print(f'TOTAL = {row_count}')

387298
380461
381438
TOTAL = 1149197


## Carregando nosso Spark DataFrame

In [627]:
findspark.init()

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('PySpark Dataframe').getOrCreate()

In [628]:
# import os
# print(os.environ['HADOOP_HOME']) 
# print(os.environ['JAVA_HOME']) 

In [629]:
df_silver = spark.read.parquet(LOCAL_SILVER)

In [630]:
df_silver.show(5)

+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+-----+-----+------+----+-----+------+------+------+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+------+-----+------+------+-----+-----+-----+------+-------+-------+------+-------+-------+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+------+-----+------+-----+-----+-----+-----+----+-----+-----+-----+-----+----+-----+-----+------+------+------+------+------+-----+----+---+
| ano| uf|capital|rm_ride|v1008|v1012|v1013|v1016|estrato|      upa|v1022|v1023| v1030|v1031|v1032|posest|a001|a001a|a

In [631]:
df_silver.createOrReplaceTempView('silver')

spark.sql('''
          SELECT b002 as estabelecimento, b0011, b0012, b0013, b0014 from silver
          WHERE (
            b0011 = 1
            OR b0012 = 1
            OR b0013 = 1
            OR b0014 = 1
            OR b0015 = 1
            OR b0016 = 1
            OR b0017 = 1
            OR b0018 = 1
            OR b0019 = 1
            OR b00110 = 1
            OR b00111 = 1
            OR b00112 = 1
            OR b00113 = 1  ) AND
          b002 = 9
          LIMIT 10
          ''').show()

+---------------+-----+-----+-----+-----+
|estabelecimento|b0011|b0012|b0013|b0014|
+---------------+-----+-----+-----+-----+
|              9|    1|    2|    2|    1|
|              9|    2|    1|    2|    2|
|              9|    2|    1|    2|    2|
|              9|    1|    2|    2|    2|
|              9|    2|    2|    1|    2|
|              9|    9|    9|    1|    9|
|              9|    9|    9|    1|    9|
|              9|    2|    2|    2|    2|
|              9|    2|    2|    2|    2|
|              9|    1|    2|    2|    1|
+---------------+-----+-----+-----+-----+



In [632]:
df_silver.groupBy('b002').count().show()

+----+-------+
|b002|  count|
+----+-------+
|NULL|1104247|
|   1|  11796|
|   9|     13|
|   2|  33141|
+----+-------+



# 3. Gold

## Manipulação

### Selecão das perguntas
Foi requesitado a utilização de no máximo 20 perguntas da pesquisa. Seguem as selecionadas abaixo.
- `UF` Unidade da Federação
 - `V1022` Situação de Domicílio
 - `A002` Qual sua idade?
 - `A004` Cor ou raça
 - `A005` Qual sua escolaridade?
 - `B001` Na semana passada, o(a) Sr(a) teve (sintomas)? 
 - `B002` Por causa (de sintomas), foi a algum estabelecimento de saúde?
 - `B004` Em que local(is) buscou o atendimento?
 - `B007` O(A) Sr(a) tem algum plano de saúde médico, seja particular, de empresa ou de órgão público?
 - `B009B` (SWAB) Qual o resultado?
 - `B009D` (Furo no dedo) Qual o resultado?
 - `B009F` (Veia no braço) Qual o resultado?
 - `B010` Algum médico já lhe deu o diagnóstico de alguma dessas doenças?
 - `B011` Na semana passada, devido à pandemia do Coronavírus, em que medida o(a) Sr(a) restringiu o contato com as pessoas?
 - `C002` Na semana passada, estava temporariamente afastado de algum trabalho?
 - `C004` O(A) Sr(a) continuou a ser remunerado (mesmo que parcialmente) por esse trabalho?
 - `C007` No trabalho (único ou principal) que tinha nessa semana, o(a) Sr(a) era:
 - `C010` Número de faixa do rendimento / retirada em dinheiro (NORMALMENTE)
 - `C011` Número de faixa do rendimento / retirada em dinheiro (NO MES ATUAL)
 - `C013` Na semana passada, o(a) Sr(a) estava em trabalho remoto (home office ou teletrabalho)? (C013)

In [693]:
selected_cols = {
    'mes':'mes',
    'uf':'uf',
    'v1022':'situacao_domicilio',
    'a002':'idade',
    'a004':'cor',
    'a005':'escolaridade',
    'b0011':'sintoma_febre',
    'b0012':'sintoma_tosse',
    'b0013':'sintoma_dor_garganta',
    'b0014':'sintoma_dificuldade_respirar',
    'b0015':'sintoma_dor_cabeca',
    'b0016':'sintoma_dor_peito',
    'b0017':'sintoma_nausea',
    'b0018':'sintoma_dor_nariz_entupido',
    'b0019':'sintoma_fadiga',
    'b00110':'sintoma_dor_olhos',
    'b00111':'sintoma_perda_cheiro_sabor',
    'b00112':'sintoma_dor_corpo',
    'b00113':'sintoma_diarreira',
    'b002':'foi_a_estabelecimento_saude',
    'b0041':'estabelecimento_posto',
    'b0042':'estabelecimento_upa',
    'b0043':'estabelecimento_hospital_sus',
    'b0044':'estabelecimento_ambulatorio_privado',
    'b0045':'estabelecimento_pronto_socorro_privado',
    'b0046':'estabelecimento_hospital_privado',
    'b007':'plano_saude',
    'b009b':'teste_resultado_swab',
    'b009d':'teste_resultado_furo',
    'b009f':'teste_resultado_braco',
    'b0101':'diagnostico_diabetes',
    'b0102':'diagnostico_hipertensao',
    'b0103':'diagnostico_respiratorias',
    'b0104':'diagnostico_cardiacas',
    'b0105':'diagnostico_depressao',
    'b0106':'diagnostico_cancer',
    'b011':'contato_reduzido',
    'c002':'afastamento_temporario_trabalho',
    'c004':'afastamento_manteu_remuneracao',
    'c007':'trabalho_funcao',
    'c01011':'rendimento_regular',
    'c011a11':'rendimento_real',
    'c013':'home_office'
}

In [694]:
df_gold = df_silver.select([c for c in selected_cols.keys()])
df_gold.show(5)

+---+---+-----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+------+-------+----+
|mes| uf|v1022|a002|a004|a005|b0011|b0012|b0013|b0014|b0015|b0016|b0017|b0018|b0019|b00110|b00111|b00112|b00113|b002|b0041|b0042|b0043|b0044|b0045|b0046|b007|b009b|b009d|b009f|b0101|b0102|b0103|b0104|b0105|b0106|b011|c002|c004|c007|c01011|c011a11|c013|
+---+---+-----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+------+-------+----+
| 10| 11|    1|  36|   4|   5|    2|    2|    2|    2|    2|    2|    2|    2|    2|     2|     2|     2|     2|NULL| NULL| NULL| NULL| NULL| NULL| NULL|   1| NULL| NULL| NULL|    2|    2|    2|    2|    2|    2|   2|NULL|NULL|   4|     4|  

In [695]:
for col in selected_cols.keys():
    df_gold = df_gold.withColumnRenamed(col, selected_cols[col])
df_gold.show(5)

+---+---+------------------+-----+---+------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+----------------+-------------------------------+------------------------------+---------------+------------------+---------------+-----------+
|mes| uf|situacao_domicilio|idade|cor|escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintom

## Criação e manipulação de colunas

In [696]:
def convert_columns(
        df,
        column_to_convert : str,
        dict_of_conversion : dict,
):
    map_col = F.create_map(
        [F.lit(x) for i in dict_of_conversion.items() for x in i]
    )

    df_new = df.withColumn(
        column_to_convert,
        map_col[F.col(column_to_convert)]
    )

    return df_new

### (NEW) Região

In [697]:
dict_regiao = {
    1: 'Norte',
    2: 'Nordeste',
    3: 'Sudeste',
    4: 'Sul',
    5: 'Centro-Oeste'
}

df_gold = df_gold.withColumn(
    'regiao',
    F.when(F.col('uf') < 21, dict_regiao[1])\
    .when(F.col('uf') < 31, dict_regiao[2])\
    .when(F.col('uf') < 41, dict_regiao[3])\
    .when(F.col('uf') < 50, dict_regiao[4])\
    .otherwise(dict_regiao[5])
    )

### (uf) UF

In [698]:
current_col = 'uf'
current_dict = dict_uf = {
    11:'Rondônia',
    12:'Acre',
    13:'Amazonas',
    14:'Roraima',
    15:'Pará',
    16:'Amapá',
    17:'Tocantins',
    21:'Maranhão',
    22:'Piauí',
    23:'Ceará',
    24:'Rio Grande do Norte',
    25:'Paraíba',
    26:'Pernambuco',
    27:'Alagoas',
    28:'Sergipe',
    29:'Bahia',
    31:'Minas Gerais',
    32:'Espírito Santo',
    33:'Rio de Janeiro',
    35:'São Paulo',
    41:'Paraná',
    42:'Santa Catarina',
    43:'Rio Grande do Sul',
    50:'Mato Grosso do Sul',
    51:'Mato Grosso',
    52:'Goiás',
    53:'Distrito Federal'
}
df_gold = convert_columns(df_gold, current_col, current_dict)

df_gold.show(3)

+---+--------+------------------+-----+---+------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+----------------+-------------------------------+------------------------------+---------------+------------------+---------------+-----------+------+
|mes|      uf|situacao_domicilio|idade|cor|escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldad

### (v1022) Situação de domicílio

In [699]:
#v1022
current_col = 'situacao_domicilio'
current_dict = {
    1: 'Urbana', 2: 'Rural'
}
df_gold = convert_columns(df_gold, current_col, current_dict)

### (a004) Cor

In [700]:
#a004
current_col = 'cor'
current_dict = {
    1: 'Branca', 2: 'Preta', 3: 'Amarela', 4: 'Parda', 5: 'Indígena', 9: None
}
df_gold = convert_columns(df_gold, current_col, current_dict)

### (a005) Escolaridade

In [701]:
#a005
current_col = 'escolaridade'
current_dict = {
    1: 'Sem instrução', 2: 'Fundamental incompleto', 3: 'Fundamental completo', 4: 'Médio incompleto', 5: 'Médio completo', 6: 'Superior incompleto', 7: 'Superior completo', 8: 'Pós-graduação, mestrado ou doutorado'
}
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b001) Sintomas

In [702]:
df_silver.groupBy('b0011').count().show()

+-----+-------+
|b0011|  count|
+-----+-------+
|    1|   9020|
|    3|    640|
|    9|   3513|
|    2|1136024|
+-----+-------+



In [703]:
# 'b0011':'sintoma_febre',
# 'b0012':'sintoma_tosse',
# 'b0013':'sintoma_dor_garganta',
# 'b0014':'sintoma_dificuldade_respirar',
# 'b0015':'sintoma_dor_cabeca',
# 'b0016':'sintoma_dor_peito',
# 'b0017':'sintoma_nausea',
# 'b0018':'sintoma_dor_nariz_entupido',
# 'b0019':'sintoma_fadiga',
# 'b00110':'sintoma_dor_olhos',
# 'b00111':'sintoma_perda_cheiro_sabor',
# 'b00112':'sintoma_dor_corpo',
# 'b00113':'sintoma_diarreira',


cols_sintomas = [
        'sintoma_febre',
        'sintoma_tosse',
        'sintoma_dor_garganta',
        'sintoma_dificuldade_respirar',
        'sintoma_dor_cabeca',
        'sintoma_dor_peito',
        'sintoma_nausea',
        'sintoma_dor_nariz_entupido',
        'sintoma_fadiga',
        'sintoma_dor_olhos',
        'sintoma_perda_cheiro_sabor',
        'sintoma_dor_corpo',
        'sintoma_diarreira', 
        ]

current_dict = {
    1: True, 2: False, 3: None, 9: None
    }
for current_col in cols_sintomas:
    df_gold = convert_columns(df_gold, current_col, current_dict)

### (NEW) Teve sintomas

In [704]:
df_gold = df_gold.withColumn(
    'teve_sintomas',
    F.when(F.col(cols_sintomas[0]) == True, True)\
    .when(F.col(cols_sintomas[1]) == True, True)\
    .when(F.col(cols_sintomas[2]) == True, True)\
    .when(F.col(cols_sintomas[3]) == True, True)\
    .when(F.col(cols_sintomas[4]) == True, True)\
    .when(F.col(cols_sintomas[5]) == True, True)\
    .when(F.col(cols_sintomas[6]) == True, True)\
    .when(F.col(cols_sintomas[7]) == True, True)\
    .when(F.col(cols_sintomas[8]) == True, True)\
    .when(F.col(cols_sintomas[9]) == True, True)\
    .when(F.col(cols_sintomas[10]) == True, True)\
    .when(F.col(cols_sintomas[11]) == True, True)\
    .when(F.col(cols_sintomas[12]) == True, True)\
    .otherwise(False)
)

df_gold.show(3)

+---+--------+------------------+-----+-----+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+----------------+-------------------------------+------------------------------+---------------+------------------+---------------+-----------+------+-------------+
|mes|      uf|situacao_domicilio|idade|  cor|        escolaridade|sintoma_febre|sintoma_tosse|sinto

In [705]:
df_gold.createOrReplaceTempView('gold')

spark.sql('''
          SELECT 
            teve_sintomas,
            sintoma_febre,
            sintoma_tosse,
            sintoma_dor_garganta,
            sintoma_dificuldade_respirar,
            sintoma_dor_peito,
            sintoma_fadiga,
            sintoma_perda_cheiro_sabor,
            sintoma_dor_corpo
          FROM gold
          WHERE (
                  sintoma_febre IS false OR
                  sintoma_tosse IS false OR
                  sintoma_dor_garganta IS false OR
                  sintoma_dificuldade_respirar IS false OR
                  sintoma_dor_peito IS false OR
                  sintoma_fadiga IS false OR
                  sintoma_perda_cheiro_sabor IS false OR
                  sintoma_dor_corpo IS false
                ) AND
                teve_sintomas IS true
          LIMIT 15
          ''').show()

+-------------+-------------+-------------+--------------------+----------------------------+-----------------+--------------+--------------------------+-----------------+
|teve_sintomas|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_peito|sintoma_fadiga|sintoma_perda_cheiro_sabor|sintoma_dor_corpo|
+-------------+-------------+-------------+--------------------+----------------------------+-----------------+--------------+--------------------------+-----------------+
|         true|        false|         true|               false|                       false|            false|         false|                     false|            false|
|         true|        false|        false|               false|                       false|            false|          true|                      true|            false|
|         true|        false|         true|                true|                       false|            false|          true|              

In [706]:
df_gold.groupBy('teve_sintomas').count().show()

+-------------+-------+
|teve_sintomas|  count|
+-------------+-------+
|         true|  44950|
|        false|1104247|
+-------------+-------+



### (b002) Foi a estabelecimento saúde

In [707]:
# b002
current_col = 'foi_a_estabelecimento_saude'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b004) Onde buscou atendimento

In [708]:
cols_estabelecimentos = [
    'estabelecimento_posto',
    'estabelecimento_upa',
    'estabelecimento_hospital_sus',
    'estabelecimento_ambulatorio_privado',
    'estabelecimento_pronto_socorro_privado',
    'estabelecimento_hospital_privado'
]

current_dict = {
    1: True,
    2: False,
    9: False
}

for current_col in cols_estabelecimentos:
    df_gold = convert_columns(df_gold, current_col, current_dict)

In [709]:
# b004
# current_dict = {
#     1: 'Posto de saúde/Unidade básica de saúde /Equipe de Saúde da Família',
#     2: 'Pronto socorro do SUS/UPA',
#     3:'Hospital do SUS',
#     4:'Ambulatório ou consultório privado ou ligado às forças armadas',
#     5: 'Pronto socorro privado ou ligado às forças armadas',
#     5: 'Hospital privado ou ligado às forças armadas',
#     9: None
#     }

df_gold = df_gold.withColumn(
    'estabelecimento_sus',
    F.when(F.col(cols_estabelecimentos[0]) == True, True)\
    .when(F.col(cols_estabelecimentos[1]) == True, True)\
    .when(F.col(cols_estabelecimentos[2]) == True,True)\
    .when(F.col(cols_estabelecimentos[3]) == True, False)\
    .when(F.col(cols_estabelecimentos[4]) == True, False)\
    .when(F.col(cols_estabelecimentos[5]) == True,False)\
    .otherwise(None)
    )


In [710]:
df_gold.createOrReplaceTempView('gold')

spark.sql('''
          SELECT 
            estabelecimento_posto,
            estabelecimento_upa,
            estabelecimento_hospital_sus,
            estabelecimento_ambulatorio_privado,
            estabelecimento_pronto_socorro_privado,
            estabelecimento_hospital_privado,
            estabelecimento_sus,
            foi_a_estabelecimento_saude
          FROM gold
          WHERE foi_a_estabelecimento_saude IS true
          LIMIT 10
          ''').show()

+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+-------------------+---------------------------+
|estabelecimento_posto|estabelecimento_upa|estabelecimento_hospital_sus|estabelecimento_ambulatorio_privado|estabelecimento_pronto_socorro_privado|estabelecimento_hospital_privado|estabelecimento_sus|foi_a_estabelecimento_saude|
+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+-------------------+---------------------------+
|                false|              false|                       false|                               true|                                 false|                           false|              false|                       true|
|                false|              false|                       false|            

In [711]:
df_gold.createOrReplaceTempView('gold')

spark.sql('''
          SELECT 
            estabelecimento_sus,
            COUNT(*),
            SUM(
              CASE
                estabelecimento_posto WHEN true then 1 else 0
              END
              ) as estabelecimento_posto,
            SUM(
              CASE
                estabelecimento_upa WHEN true then 1 else 0
              END
              ) as estabelecimento_upa,
            SUM(
              CASE
                estabelecimento_hospital_sus WHEN true then 1 else 0
              END
              ) as estabelecimento_hospital_sus,
            SUM(
              CASE
                estabelecimento_ambulatorio_privado WHEN true then 1 else 0
              END
              ) as estabelecimento_ambulatorio_privado,
            SUM(
              CASE
                estabelecimento_pronto_socorro_privado WHEN true then 1 else 0
              END
              ) as estabelecimento_pronto_socorro_privado,
            SUM(
              CASE
                estabelecimento_hospital_privado WHEN true then 1 else 0
              END
              ) as estabelecimento_hospital_privado
          FROM gold
          GROUP BY estabelecimento_sus
          LIMIT 10
          ''').show()

+-------------------+--------+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+
|estabelecimento_sus|count(1)|estabelecimento_posto|estabelecimento_upa|estabelecimento_hospital_sus|estabelecimento_ambulatorio_privado|estabelecimento_pronto_socorro_privado|estabelecimento_hospital_privado|
+-------------------+--------+---------------------+-------------------+----------------------------+-----------------------------------+--------------------------------------+--------------------------------+
|               NULL| 1137622|                    0|                  0|                           0|                                  0|                                     0|                               0|
|               true|    9256|                 5591|               2419|                        2124|                                135|                       

In [712]:
df_gold = df_gold.drop(
    *cols_estabelecimentos
)

### (b007) Tem plano de saúde

In [744]:
df_silver.groupBy('b007').count().show()

+----+------+
|b007| count|
+----+------+
|   1|260528|
|   9|  2470|
|   2|886199|
+----+------+



In [713]:
# b007
current_col = 'plano_saude'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b009B) Teste SWAB

In [714]:
# b009b
current_col = 'teste_resultado_swab'
current_dict = {
    1: 'Positivo', 2: 'Negativo', 3: 'Inconclusivo', 4: 'Não recebeu', 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b009D) Teste de furo no dedo

In [715]:
# b009d
current_col = 'teste_resultado_furo'
current_dict = {
    1: 'Positivo', 2: 'Negativo', 3: 'Inconclusivo', 4: 'Não recebeu', 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b009F) Teste de sangue do braço

In [716]:
# b009d
current_col = 'teste_resultado_braco'
current_dict = {
    1: 'Positivo', 2: 'Negativo', 3: 'Inconclusivo', 4: 'Não recebeu', 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b010) Diagnósticos
#### Diagnóstico de diabetes

In [717]:
# b0101
current_col = 'diagnostico_diabetes'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

#### Diagnóstico hipertensão

In [718]:
# b0102
current_col = 'diagnostico_hipertensao'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

#### Diagnóstico de doenças respiratórias

In [719]:
# b0103
current_col = 'diagnostico_respiratorias'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

#### Diagnóstico de doenças cardíacas

In [720]:
# b0104
current_col = 'diagnostico_cardiacas'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

#### Diagnóstico de depressão

In [721]:
# b0105
current_col = 'diagnostico_depressao'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

#### Diagnóstico de câncer

In [722]:
# b0106
current_col = 'diagnostico_cancer'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (b011) Contato reduzido

In [723]:
# b011
current_col = 'contato_reduzido'
current_dict = {
    1: 'Sem restrição', 2: 'Reduziu mas continuou saindo', 3: 'Só saiu em caso de necessidade', 4: 'Ficou rigorosamente em casa'
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (c002) Estava temporariamente afastado de algum trabalho

In [724]:
df_gold.groupBy('afastamento_temporario_trabalho').count().show()

+-------------------------------+------+
|afastamento_temporario_trabalho| count|
+-------------------------------+------+
|                           NULL|624881|
|                              1| 41605|
|                              2|482711|
+-------------------------------+------+



In [725]:
# c002
current_col = 'afastamento_temporario_trabalho'
current_dict = {
    1: True, 2: False, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (c004) Afastamento foi remunerado

In [726]:
df_gold.groupBy('afastamento_manteu_remuneracao').count().show()

+------------------------------+-------+
|afastamento_manteu_remuneracao|  count|
+------------------------------+-------+
|                          NULL|1123153|
|                             1|  11738|
|                             3|    222|
|                             2|  14084|
+------------------------------+-------+



In [727]:
# c004
current_col = 'afastamento_manteu_remuneracao'
current_dict = {
    1: True, 2: False, 3: True, 9: None
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

In [728]:
df_gold.groupBy('afastamento_manteu_remuneracao').count().show()

+------------------------------+-------+
|afastamento_manteu_remuneracao|  count|
+------------------------------+-------+
|                          NULL|1123153|
|                          true|  11960|
|                         false|  14084|
+------------------------------+-------+



### (c007) Função no trabalho

In [729]:
# c007c
current_col = 'trabalho_funcao'
current_dict = {
    1:'Empregado doméstico, diarista, cozinheiro (em domicílios particulares), ',
    2:'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada), ',
    3:'Auxiliar de escritório, escriturário',
    4:'Secretária, recepcionista',
    5:'Operador de Telemarketing',
    6:'Comerciante (dono do bar, da loja etc.) ',
    7:'Balconista, vendedor de loja',
    8:'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)  ',
    9:'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)',
    10:'Cozinheiro e garçom (de restaurantes, empresas)',
    11:'Padeiro, açougueiro e doceiro ',
    12:'Agricultor, criador de animais, pescador, silvicultor e jardineiro',
    13:'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)',
    14:'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)',
    15:'Motorista de caminhão (caminhoneiro), ',
    16:'Motoboy, ',
    17:'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.) ',
    18:'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro',
    19:'Mecânico de veículos, máquinas industriais etc.',
    20:'Artesão, costureiro e sapateiro',
    21:'Cabeleireiro, manicure e afins',
    22:'Operador de máquinas, montador na indústria; ',
    23:'Auxiliar de produção, de carga e descarga;',
    24:'Professor da educação infantil, de ensino fundamental, médio ou superior,',
    25:'Pedagogo, professor de idiomas, música, arte e reforço escolar',
    26:'Médico, enfermeiro, profissionais de saúde de nível superior',
    27:'Técnico, profissional da saúde de nível médio ',
    28:'Cuidador de crianças, doentes ou idosos',
    29:'Segurança, vigilante, outro trabalhador dos serviços de proteção',
    30:'Policial civil',
    31:'Porteiro, zelador',
    32:'Artista, religioso (padre, pastor etc.)',
    33:'Diretor, gerente, cargo político ou comissionado',
    34:'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)',
    35:'Outro técnico ou profissional de nível médio',
    36:'Outros'
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (c010) Rendimento regular

In [730]:
# c01011
current_col = 'rendimento_regular'
current_dict = {
    0 : '0 - 100',
    1 : '101 - 300',
    2 : '301 - 600',
    3 : '601 - 800',
    4 : '801 - 1.600',
    5 : '1.601 - 3.000',
    6 : '3.001 - 10.000',
    7 : '10.001 - 50.000',
    8 : '50.001 - 100.000',
    9 : 'Mais de 100.000'
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (c011) Rendimento real

In [731]:
# c011a11
current_col = 'rendimento_real'
current_dict = {
    0 : '0 - 100',
    1 : '101 - 300',
    2 : '301 - 600',
    3 : '601 - 800',
    4 : '801 - 1.600',
    5 : '1.601 - 3.000',
    6 : '3.001 - 10.000',
    7 : '10.001 - 50.000',
    8 : '50.001 - 100.000',
    9 : 'Mais de 100.000'
    }
df_gold = convert_columns(df_gold, current_col, current_dict)

### (NEW) Rendimento mudou?

In [732]:
df_gold = df_gold.withColumn(
    'rendimento_mudou',
    F.when(F.col('rendimento_regular') == F.col('rendimento_real'), False)\
    .when(F.col('rendimento_regular').isNull(), False)\
    .when(F.col('rendimento_real').isNull(), False).otherwise(True)
)

df_gold.show()

+---+--------+------------------+-----+-------+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+
|mes|      uf|situacao_domicilio|idade|    cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_dor_nariz_entupido|sinto

### (c013) Trabalhou em home office

In [733]:
# c0013
current_col = 'home_office'
df_gold.na.fill({current_col : False})
current_dict = {
    1: True, 2: False
    }

df_gold = convert_columns(df_gold, current_col, current_dict)

### (NEW) Fez teste

In [734]:
df_gold = df_gold.withColumn(
    'teste_feito',
    F.when(F.col('teste_resultado_swab').isNull() & F.col('teste_resultado_furo').isNull() & F.col('teste_resultado_braco').isNull(), False)\
    .otherwise(True)
)

df_gold.show(3)

+---+--------+------------------+-----+-----+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+
|mes|      uf|situacao_domicilio|idade|  cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_dor_nariz_entupi

### (NEW) Teste positivo

In [735]:
df_gold = df_gold.withColumn(
    'teste_positivo',
    F.when(F.col('teste_resultado_swab') == 'Positivo', True)\
    .when(F.col('teste_resultado_furo') == 'Positivo', True)\
    .when(F.col('teste_resultado_braco') == 'Positivo',True)\
    .otherwise(False)
    )

df_gold.show(3)

+---+--------+------------------+-----+-----+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+--------------------+---------------------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+--------------+
|mes|      uf|situacao_domicilio|idade|  cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_d

In [736]:
df_gold.createOrReplaceTempView('gold')

spark.sql('''
          SELECT teste_resultado_swab, teste_resultado_furo, teste_resultado_braco, teste_positivo, teste_feito FROM gold
          WHERE teste_resultado_swab = 'Positivo' OR teste_resultado_furo = 'Positivo' OR teste_resultado_braco = 'Positivo'
          ''').show()

+--------------------+--------------------+---------------------+--------------+-----------+
|teste_resultado_swab|teste_resultado_furo|teste_resultado_braco|teste_positivo|teste_feito|
+--------------------+--------------------+---------------------+--------------+-----------+
|            Positivo|            Negativo|             Positivo|          true|       true|
|                NULL|            Positivo|             Positivo|          true|       true|
|                NULL|            Positivo|             Positivo|          true|       true|
|                NULL|                NULL|             Positivo|          true|       true|
|        Inconclusivo|                NULL|             Positivo|          true|       true|
|            Positivo|                NULL|             Negativo|          true|       true|
|            Positivo|            Positivo|             Positivo|          true|       true|
|                NULL|            Positivo|             Positivo|     

In [737]:
df_gold.createOrReplaceTempView('gold')

spark.sql('''
          SELECT teste_resultado_swab, teste_resultado_furo, teste_resultado_braco, teste_positivo, teste_feito FROM gold
          WHERE teste_resultado_swab = 'Negativo' OR teste_resultado_furo = 'Negativo' OR teste_resultado_braco = 'Negativo'
          ''').show()

+--------------------+--------------------+---------------------+--------------+-----------+
|teste_resultado_swab|teste_resultado_furo|teste_resultado_braco|teste_positivo|teste_feito|
+--------------------+--------------------+---------------------+--------------+-----------+
|                NULL|            Negativo|             Negativo|         false|       true|
|            Positivo|            Negativo|             Positivo|          true|       true|
|            Negativo|                NULL|                 NULL|         false|       true|
|            Negativo|                NULL|                 NULL|         false|       true|
|            Positivo|                NULL|             Negativo|          true|       true|
|                NULL|            Negativo|             Positivo|          true|       true|
|            Negativo|            Negativo|             Negativo|         false|       true|
|                NULL|                NULL|             Negativo|     

In [738]:
df_gold = df_gold.drop(
    *['teste_resultado_swab', 'teste_resultado_furo', 'teste_resultado_braco']
)

df_gold.show(3)

+---+--------+------------------+-----+-----+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+--------------+
|mes|      uf|situacao_domicilio|idade|  cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_dor_nariz_entupido|sintoma_fadiga|sintoma_dor_olhos|sintoma_perda

### (NEW) Acima de 14 anos

In [739]:
df_gold = df_gold.withColumn(
    'acima_14',
    F.when(F.col('idade') >= 14, True).otherwise(False)
)

df_gold.show(5)

+---+--------+------------------+-----+------+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+--------------+--------+
|mes|      uf|situacao_domicilio|idade|   cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_dor_nariz_entupido|sintoma_fadiga|sintoma_dor_olhos|si

## Final

In [619]:
df_gold.printSchema()

root
 |-- mes: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- situacao_domicilio: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- cor: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- sintoma_febre: boolean (nullable = true)
 |-- sintoma_tosse: boolean (nullable = true)
 |-- sintoma_dor_garganta: boolean (nullable = true)
 |-- sintoma_dificuldade_respirar: boolean (nullable = true)
 |-- sintoma_dor_peito: boolean (nullable = true)
 |-- sintoma_fadiga: boolean (nullable = true)
 |-- sintoma_perda_cheiro_sabor: boolean (nullable = true)
 |-- sintoma_dor_corpo: boolean (nullable = true)
 |-- foi_a_estabelecimento_saude: boolean (nullable = true)
 |-- plano_saude: boolean (nullable = true)
 |-- diagnostico_diabetes: boolean (nullable = true)
 |-- diagnostico_hipertensao: boolean (nullable = true)
 |-- diagnostico_respiratorias: boolean (nullable = true)
 |-- diagnostico_cardiacas: boolean (nullable = true)
 |-- diagnostico_depr

In [620]:
df_gold.show()

+---+--------+------------------+-----+-------+--------------------+-------------+-------------+--------------------+----------------------------+-----------------+--------------+--------------------------+-----------------+---------------------------+-----------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+--------------+--------+
|mes|      uf|situacao_domicilio|idade|    cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_peito|sintoma_fadiga|sintoma_perda_cheiro_sabor|sintoma_dor_corpo|foi_a_estabelecimento_saude|plano_saude|diagnostico_diabetes|diagnostico_hipertensao|diagnostico_respiratorias|diagnostico_cardiaca

## Exportando nossa camada Gold

### Local

In [740]:
df_gold.write.parquet('local/gold/gold.parquet', mode = 'overwrite')

print(df_gold.count())

1149197


In [741]:
df_gold = spark.read.parquet(f'{LOCAL_GOLD}/gold.parquet')

print(df_gold.count())
df_gold.show(3)

1149197
+---+--------+------------------+-----+-----+--------------------+-------------+-------------+--------------------+----------------------------+------------------+-----------------+--------------+--------------------------+--------------+-----------------+--------------------------+-----------------+-----------------+---------------------------+-----------+--------------------+-----------------------+-------------------------+---------------------+---------------------+------------------+--------------------+-------------------------------+------------------------------+--------------------+------------------+---------------+-----------+------+-------------+-------------------+----------------+-----------+--------------+--------+
|mes|      uf|situacao_domicilio|idade|  cor|        escolaridade|sintoma_febre|sintoma_tosse|sintoma_dor_garganta|sintoma_dificuldade_respirar|sintoma_dor_cabeca|sintoma_dor_peito|sintoma_nausea|sintoma_dor_nariz_entupido|sintoma_fadiga|sintoma_dor_ol

### Nuvem

In [105]:
for root, folders, files in os.walk(f'{LOCAL_GOLD}'):
    for file in files:
        if file.endswith('.parquet'):
            file_path = os.path.join(root, file)
            print(f'Attempting to upload {file_path}')
            try:
                upload_to_s3(
                    file_path,
                    BUCKET,
                    f'{GOLD}/{file}')
            except Exception as e:
                print(f'Failed to upload | {e}')

Attempting to upload ./local/gold\gold.parquet\part-00000-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet
Attempting to upload ./local/gold\gold.parquet\part-00001-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet
Attempting to upload ./local/gold\gold.parquet\part-00002-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet
Attempting to upload ./local/gold\gold.parquet\part-00003-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet
Attempting to upload ./local/gold\gold.parquet\part-00004-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet
Attempting to upload ./local/gold\gold.parquet\part-00005-4dd2e73a-1899-4c11-ba4c-b5bf4b76d1a6-c000.snappy.parquet


## Conectando com o PostgreSQL

In [ ]:
usuario = 'postgres'
senha = ''
with open('./ignore/postgresql-pw') as f:
    senha = f.read()
host = 'bd-relacional.circagi8eu0o.us-east-1.rds.amazonaws.com'
porta = 5432
banco = 'postgres'

engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}",
                       echo = True)


In [ ]:
def test_connection(engine):
    try:
        with engine.connect() as connection:
            # Testa a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Lista as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)


In [ ]:
test_connection(engine)

2025-09-21 16:54:21,115 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-09-21 16:54:21,115 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 16:54:21,396 INFO sqlalchemy.engine.Engine select current_schema()
2025-09-21 16:54:21,397 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 16:54:21,682 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-09-21 16:54:21,684 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-09-21 16:54:21,970 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 16:54:21,973 INFO sqlalchemy.engine.Engine SELECT version();
2025-09-21 16:54:21,976 INFO sqlalchemy.engine.Engine [generated in 0.00530s] {}
✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
2025-09-21 16:54:22,255 INFO sqlalchemy.engine.Engine 
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            
2025-09-21 16:54:22,256 INFO s

### Inserindo nossos dados

In [ ]:
df = pd.read_parquet(LOCAL_GOLD)

df.shape

(1149197, 31)

In [ ]:
df.to_sql(
    name = 'pnad',
    con = engine,
    if_exists = 'append',
    index = False,
    method = 'multi',
    chunksize = 100_000
)

2025-09-21 16:54:23,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-21 16:54:23,346 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-09-21 16:54:23,346 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {'table_name': 'pnad', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
